In [1]:
import moviepy.editor as mp
from pydub import AudioSegment
from pydub.silence import split_on_silence
from vosk import Model, KaldiRecognizer
import subprocess
from pathlib import Path
import numpy as np

# Step 1: Extract audio from video
video = mp.VideoFileClip("um.mp4")
audio = video.audio


# Step 2: Convert audio to WAV for speech recognition
audio.write_audiofile("audio.wav", codec="pcm_s16le")

# Step 3: Speech recognition using Vosk (you'll need to install the vosk library)
model_path = "../vosk-model-en-us-0.22"
try:
    model = Model(model_path)
except Exception as e:
    print(f"Error: {e}")
    # Add additional information or actions as needed

if 'model' in locals():
    # Continue with the rest of the code only if 'model' is defined

    audio_data = AudioSegment.from_wav("audio.wav")
    chunks = split_on_silence(audio_data, silence_thresh=-40)
    recognized_text = ""

    for chunk in chunks:
        # Convert audio data to numpy array of 16-bit signed integers
        audio_np = np.fromstring(chunk.raw_data, dtype=np.int16)
        recognizer = KaldiRecognizer(model, audio_np)
        recognized_text += recognizer.FinalResult()

    # Rest of the code...
else:
    print("Model not available, cannot proceed.")


# Step 4: Identify "um" occurrences and cut corresponding video segments
um_segments = [(start, end) for start, end in zip(chunks[:-1], chunks[1:]) if "um" in recognized_text[start:end]]

edited_segments = []
for start, end in um_segments:
    edited_segments.append(video.subclip(start, end))

final_video = mp.concatenate_videoclips(edited_segments)

# Step 5: Export final video
final_video.write_videofile("umRemove.mp4")


MoviePy - Writing audio in audio.wav


MoviePy - Done.


TypeError: only length-1 arrays can be converted to Python scalars